<h1 id="Contents">Contents<a href="#Contents"></a></h1>
        <ol>
        <li><a class="" href="#Forward">Forward</a></li>
<li><a class="" href="#Future">Future</a></li>
<ol><li><a class="" href="#Future-Price-Formula">Future Price Formula</a></li>
</ol>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import glob

import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "notebook"
pio.templates.default = "plotly_dark"

# Forward

# Future

## Future Price Formula

The future price of a stock is the price of the stock at a future date. The future price of a stock is calculated by using the following formula:
$$
F = S \times (1+R_f*T) - D_1
$$
where:
* $F$ is the future price of the stock
* $S$ is the current price of the stock
* $R_f$ is the risk-free rate of return
* $D_1$ is the dividend paid on the stock
* $T$ is the time to maturity

In [2]:
nifty = pd.read_csv("option_data/NIFTY.csv", parse_dates=["Date"], index_col="Date")
future = pd.read_csv('option_data/final/NIFTY_Futures_29-Dec-2022.csv', parse_dates=["Date"], index_col="Date")
bond = pd.read_csv('option_data/10Y.csv', parse_dates=["Date"], index_col="Date")

We'll be working with just the `Close` column. In India, risk-free rate of return is 7.3% (as of Jan 2023). We'll assume that there is no dividend paid on the stock.

In [3]:
nifty = nifty["Close"]
future = future["Close"]
bond = bond["Close"]

merged = pd.merge(nifty, future, left_index=True, right_index=True, how="inner")
merged = pd.merge(merged, bond, left_index=True, right_index=True, how="inner")

merged.columns = ["Nifty", "Future", "Bond"]
merged.head()

,Nifty,Future,Bond
Date,,,
2022-12-27,18132.30,18138.00,7.310
2022-12-26,18014.60,18027.30,7.319
2022-12-23,17806.80,17863.90,7.318
2022-12-22,18127.35,18170.35,7.316
2022-12-21,18199.10,18275.90,7.287


In [4]:
merged["Bond"] = merged["Bond"]/100/365

In [9]:
merged["Days_remaining"] = -(merged.index - merged.index[0]).days
# merged["Days_remaining"] = np.arange(0, len(merged))
merged["Days_remaining"] = merged["Days_remaining"].astype(int)
merged.head()

,Nifty,Future,Bond,Days_remaining,F_Simple,F_Compound
Date,,,,,,
2022-12-27,18132.30,18138.00,0.000200,0,18132.300000,18132.300000
2022-12-26,18014.60,18027.30,0.000201,1,18018.212297,18018.212297
2022-12-23,17806.80,17863.90,0.000200,2,17821.080566,17821.084861
2022-12-22,18127.35,18170.35,0.000200,3,18145.517081,18145.524365
2022-12-21,18199.10,18275.90,0.000200,4,18220.900029,18220.910912


In [10]:
merged["F_Simple"] = merged["Nifty"] * (1+merged["Bond"] * merged["Days_remaining"])
merged["F_Compound"] = merged["Nifty"] * (1+merged["Bond"])**merged["Days_remaining"]

In [11]:
merged.head()

,Nifty,Future,Bond,Days_remaining,F_Simple,F_Compound
Date,,,,,,
2022-12-27,18132.30,18138.00,0.000200,0,18132.300000,18132.300000
2022-12-26,18014.60,18027.30,0.000201,1,18018.212297,18018.212297
2022-12-23,17806.80,17863.90,0.000200,2,17813.940283,17813.940999
2022-12-22,18127.35,18170.35,0.000200,3,18138.250249,18138.252434
2022-12-21,18199.10,18275.90,0.000200,4,18213.633353,18213.637705


Let's plot the historical future price of the stock and the calculated future price of the stock.

In [12]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=merged.index, y=merged["Nifty"], name="Nifty"))
fig.add_trace(go.Scatter(x=merged.index, y=merged["Future"], name="Future"))
fig.add_trace(go.Scatter(x=merged.index, y=merged["F_Compound"], name="F_Compound"))
fig.add_trace(go.Scatter(x=merged.index, y=merged["F_Simple"], name="F_Simple"))
fig.show()


We see that the calculated future price of the stock is very close to the historical future price of the stock. Though the calculated value is slightly lower than the historical value. This can be attributed to the fact that there is also a price due to supply and demand of the stock.